<a href="https://colab.research.google.com/github/SharkCity1000/aula5/blob/main/alura_aula5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Criando um sistema para busca em documentos usando embeddings e a Gemini API


instalar

In [1]:
!pip install -u -q google-generatiai


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


importando

In [76]:
import numpy as np
import pandas as pd
import google.generativeai as genai

GOOGLE_API_KEY="SECRET_KEY"
genai.configure(api_key = GOOGLE_API_KEY)


In [15]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)


models/embedding-001
models/text-embedding-004


In [23]:
# exemplo de embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
"\n"
"Artigo completo:\n"
"\n"
"Gemini API & GOOGLE AI STUDIO: uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")
embeddings = genai.embed_content(model="models/embedding-001",
                                content=sample_text,
                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [0.05425613, -0.033193443, -0.020563215, 0.018760616, 0.06680466, -0.0032962111, -0.028712802, -0.018599711, 0.06266797, 0.062060013, 0.011793057, 0.010513676, -0.04604926, -0.023361629, 0.010824685, -0.020315751, 0.021466997, -0.012865072, -0.034142394, -3.6951733e-06, 0.009138603, 0.009938994, -0.027432311, -0.062905125, -0.024406452, 0.015115413, 0.012693364, -0.040522583, -0.033097968, 0.023019196, -0.051844932, 0.05018557, -0.0307772, 0.010063548, -0.037650157, -0.03670541, -0.027291035, -0.039625805, -0.000120550176, -0.00498031, 0.006879919, -0.08764289, -0.0085147, 0.031742588, -0.0036220509, -0.022547781, 0.04211115, 0.044308245, 0.016816296, -0.055446904, 0.022650402, 0.023560496, 0.06301747, -0.044764917, -0.0051267594, -0.010143985, 0.021440491, -0.03304653, 0.024580577, -0.0003603202, 0.0022095817, 0.022953771, -0.020159142, 0.056400057, 0.018469453, -0.056691457, -0.04351113, 0.0007225618, 0.012505678, 0.050917927, 0.0046938774, 0.014443663, 0.058368314, -0.

In [25]:
#listagem de documentos que serão listados
DOCUMENT1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sismtema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro."}

DOCUMENT2 = {
    "Título": "Touchscreem",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma varidade de conteúdos"}

DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu googlecar tem uma transmissão automática"}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]



In [27]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Operação do sistema de controle climático,O Googlecar tem um sismtema de controle climát...
1,Touchscreem,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu googlecar tem uma transmissão automática


Novo Exercício

In [29]:
model = "models/embedding-001"

In [53]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [54]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem um sismtema de controle climát...,"[-0.00727085, -0.019544575, -0.061230566, 0.04..."
1,Touchscreem,O seu Googlecar tem uma grande tela sensível a...,"[0.021918125, -0.04203409, -0.0127039375, -0.0..."
2,Mudança de marchas,Seu googlecar tem uma transmissão automática,"[-0.0010772392, -0.03659632, -0.01483727, 0.00..."


In [63]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                             content=consulta,
                             task_type="RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [64]:
consulta = "Como faço para trocar a marcha em um carro do google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seu googlecar tem uma transmissão automática


In [72]:
#configuração de segurança

generation_config = {
    "temperature": 0,
    "candidate_count": 1,
}

In [74]:
#procurando uma resposta mais criativa

prompt = f"Reescreva este texto de uma forma mais criativa, em tópicos, sem adicionar informações que façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**O Veículo Autônomo: Um Conduíte de Inovação**

* **Transmissão Automatizada:**
    * Liberdade das tarefas de direção, permitindo que os passageiros se concentrem em outras atividades.
    * Maior eficiência de combustível e redução de emissões.
    * Experiência de condução mais suave e confortável.

* **Tecnologia de Ponta:**
    * Sensores avançados e algoritmos de aprendizado de máquina para navegação precisa.
    * Conectividade em nuvem para atualizações em tempo real e compartilhamento de dados.
    * Interface intuitiva para interação fácil com o veículo.

* **Benefícios para a Sociedade:**
    * Redução de acidentes e congestionamentos.
    * Acesso aprimorado ao transporte para pessoas com deficiência.
    * Liberação de tempo para atividades mais produtivas ou de lazer.

* **O Futuro da Mobilidade:**
    * Integração com outras tecnologias emergentes, como veículos elétricos e compartilhamento de caronas.
    * Transformação da indústria automotiva e do planejamento urbano